In [61]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
import math

from pyvrp import Model, Solution
from pyvrp.plotting import plot_coordinates, plot_solution
from pyvrp.stop import MaxRuntime

In [62]:

COORDS = [
    (456, 320),  # location 0 - the depot
    (228, 0),    # location 1
    (912, 0),    # location 2
    (0, 80),     # location 3
    (67, 24),    # location 4
    (234, 145),  # location 5
    (113, 221),  # location 6
    (380, 200)  # location 7
    ]
DEMANDS = [0, 1, 1, 1, 1, 1, 1, 1]



In [63]:

DURATION_MATRIX = [
        [0, 1, 1, 1, 1, 1, 1, 1],
        [1, 0, 1, 1, 1, 1, 1, 1],
        [1, 1, 0, 1, 1, 1, 1, 1],
        [1, 1, 1, 0, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 1],
        [1, 1, 1, 1, 1, 1, 1, 0]
]
TIME_WINDOWS = [
        (8, 18),  # location 0 - the depot (modified to be unrestricted)
        (8, 12),   # location 1
        (12, 14),  # location 2
        (14, 16),  # location 3
        (8, 10),   # location 4
        (10, 12),  # location 5
        (12, 14),  # location 6
        (14, 16)   # location 7
]



In [64]:
Number_of_vehicles = 1
Vehicle_capacity = 10
service_time = 0


In [65]:
customers = set(range(1, len(COORDS)))

In [66]:
def all_subsets(s):
    s = list(s)
    return [set(combo) for r in range(len(s)+1) for combo in itertools.combinations(s, r)]

print(all_subsets(customers))


[set(), {1}, {2}, {3}, {4}, {5}, {6}, {7}, {1, 2}, {1, 3}, {1, 4}, {1, 5}, {1, 6}, {1, 7}, {2, 3}, {2, 4}, {2, 5}, {2, 6}, {2, 7}, {3, 4}, {3, 5}, {3, 6}, {3, 7}, {4, 5}, {4, 6}, {4, 7}, {5, 6}, {5, 7}, {6, 7}, {1, 2, 3}, {1, 2, 4}, {1, 2, 5}, {1, 2, 6}, {1, 2, 7}, {1, 3, 4}, {1, 3, 5}, {1, 3, 6}, {1, 3, 7}, {1, 4, 5}, {1, 4, 6}, {1, 4, 7}, {1, 5, 6}, {1, 5, 7}, {1, 6, 7}, {2, 3, 4}, {2, 3, 5}, {2, 3, 6}, {2, 3, 7}, {2, 4, 5}, {2, 4, 6}, {2, 4, 7}, {2, 5, 6}, {2, 5, 7}, {2, 6, 7}, {3, 4, 5}, {3, 4, 6}, {3, 4, 7}, {3, 5, 6}, {3, 5, 7}, {3, 6, 7}, {4, 5, 6}, {4, 5, 7}, {4, 6, 7}, {5, 6, 7}, {1, 2, 3, 4}, {1, 2, 3, 5}, {1, 2, 3, 6}, {1, 2, 3, 7}, {1, 2, 4, 5}, {1, 2, 4, 6}, {1, 2, 4, 7}, {1, 2, 5, 6}, {1, 2, 5, 7}, {1, 2, 6, 7}, {1, 3, 4, 5}, {1, 3, 4, 6}, {1, 3, 4, 7}, {1, 3, 5, 6}, {1, 3, 5, 7}, {1, 3, 6, 7}, {1, 4, 5, 6}, {1, 4, 5, 7}, {1, 4, 6, 7}, {1, 5, 6, 7}, {2, 3, 4, 5}, {2, 3, 4, 6}, {2, 3, 4, 7}, {2, 3, 5, 6}, {2, 3, 5, 7}, {2, 3, 6, 7}, {2, 4, 5, 6}, {2, 4, 5, 7}, {2, 4, 6, 7}

In [67]:
def relax_time_windows(time_windows_):
    time_windows__ = time_windows_.copy()
    return [(8, 18) for early, late in time_windows__]
def relax_client_time_windows(time_windows_, client):
    time_windows__ = time_windows_.copy()
    time_windows__[client] = (8, 18)
    return time_windows__
def relax_clients_time_windows(time_windows_, clients):
    time_windows__ = time_windows_.copy()
    for client in clients:
        time_windows__[client] = (8, 18)
    return time_windows__
def relax_time_windows_but_clients(time_windows_, clients):
    time_windows__ = time_windows_.copy()
    clients_ = list(set(range(len(time_windows__))) - set(clients))
    for client in clients_:
        time_windows__[client] = (8, 18)
    return time_windows__

print(TIME_WINDOWS)
print(relax_time_windows(TIME_WINDOWS))
print(relax_client_time_windows(TIME_WINDOWS, 1))
print(relax_clients_time_windows(TIME_WINDOWS, {1, 2}))

print(relax_time_windows_but_clients(TIME_WINDOWS, {1, 2}))


print(len(customers))
print(customers)

[(8, 18), (8, 12), (12, 14), (14, 16), (8, 10), (10, 12), (12, 14), (14, 16)]
[(8, 18), (8, 18), (8, 18), (8, 18), (8, 18), (8, 18), (8, 18), (8, 18)]
[(8, 18), (8, 18), (12, 14), (14, 16), (8, 10), (10, 12), (12, 14), (14, 16)]
[(8, 18), (8, 18), (8, 18), (14, 16), (8, 10), (10, 12), (12, 14), (14, 16)]
[(8, 18), (8, 12), (12, 14), (8, 18), (8, 18), (8, 18), (8, 18), (8, 18)]
7
{1, 2, 3, 4, 5, 6, 7}


In [ ]:
def coalition_cost(time_windows_, clients):
    
    TIME_WINDOWS_LOCAL = relax_time_windows_but_clients(time_windows_, clients)
    m1 = Model()
    m1.add_vehicle_type(
        Number_of_vehicles,
        capacity=Vehicle_capacity,
        max_duration=1000,
        tw_early=TIME_WINDOWS_LOCAL[0][0],
        tw_late=TIME_WINDOWS_LOCAL[0][1],
        )
    depot = m1.add_depot(x=COORDS[0][0], y=COORDS[0][1])
    clients = [
        m1.add_client(
        x=COORDS[idx][0],
        y=COORDS[idx][1],
        delivery=DEMANDS[idx],
        service_duration=service_time,
        tw_early=TIME_WINDOWS_LOCAL[idx][0],
        tw_late=TIME_WINDOWS_LOCAL[idx][1],
        )
        for idx in range(1, len(COORDS))
    ]
    locations = [depot] + clients
    distance_matrix = [[0] * len(locations) for _ in range(len(locations))]

    for frm_idx, frm in enumerate(locations):
        for to_idx, to in enumerate(locations):
            distance = abs(frm.x - to.x) + abs(frm.y - to.y)  # Manhattan
            duration = DURATION_MATRIX[frm_idx][to_idx]
            m1.add_edge(frm, to, distance=distance, duration=duration)
            distance_matrix[frm_idx][to_idx] = abs(frm.x - to.x) + abs(frm.y - to.y)  # Manhattan distance

    res = m1.solve(stop=MaxRuntime(1), display=False)
    return res.cost() 
    



In [69]:
def shapley_value_TW():
    n = len(COORDS) -1
    shapley_alloc = {i: 0 for i in customers}
    for i in customers:
        subsets = all_subsets(customers - {i})
        for S in subsets:
            weight = math.factorial(len(S)) * math.factorial(n - len(S) - 1) / math.factorial(n)
            marginal = coalition_cost(TIME_WINDOWS,sorted(S.union({i}))) - coalition_cost(TIME_WINDOWS,S)
            shapley_alloc[i] += weight * marginal
    return shapley_alloc
print(shapley_value_TW())

{1: 39.447619047619035, 2: 344.347619047619, 3: 193.01428571428573, 4: 298.04761904761915, 5: 115.5142857142857, 6: 149.74761904761903, 7: 21.88095238095238}
